# Multivariate Time Series Forecasting with Transformers 
Dataset: http://bit.ly/covid19-poland

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import networkx
import seaborn
import sklearn
from tabulate import tabulate

## Load dataset

In [19]:
df = pd.read_csv('DataSets/COVID-19_w_Polsce-Wzrost.csv', sep=';', usecols=["Data", "Nowe przypadki", "Zmiana liczby aktywnych przypadków"])
print(df.head())
print(df.tail())
print(f"Number of records: {len(df)}")
print("Labels types:")
print(f"{df.dtypes}")

# table = df.values.tolist()
# headers = df.columns.tolist()
# print(tabulate(table, headers=headers, tablefmt="pretty"))

# styled_df = df.style.format('{:^10}').set_properties(**{'border': '1px solid black', 'text-align': 'center'}).set_table_styles([{'selector': 'th', 'props': [('border', '1px solid black')]}])
# styled_df

   Data Nowe przypadki Zmiana liczby aktywnych przypadków
0  3.03            + 0                                + 0
1  4.03            + 1                                + 1
2  5.03            + 0                                + 0
3  6.03            + 4                                + 4
4  7.03            + 1                                + 1
      Data Nowe przypadki Zmiana liczby aktywnych przypadków
738  11.03        + 11637                              2 917
739  12.03            NaN                          5 169 557
740    NaN            NaN                                NaN
741    NaN            NaN                                NaN
742    NaN            NaN                                NaN
Number of records: 743
Labels types:
Data                                  float64
Nowe przypadki                         object
Zmiana liczby aktywnych przypadków     object
dtype: object


## Data preprocessing

### Remove NaNs

In [20]:
df = df.dropna()
# for index, row in df.iterrows():
#     if pd.isna(row['Data']):
#         df.drop(index, inplace=True)
print(f"Number of records after dropping all NaNs: {len(df)}")
df

Number of records after dropping all NaNs: 739


,Data,Nowe przypadki,Zmiana liczby aktywnych przypadków
0,3.03,+ 0,+ 0
1,4.03,+ 1,+ 1
2,5.03,+ 0,+ 0
3,6.03,+ 4,+ 4
4,7.03,+ 1,+ 1
...,...,...,...
734,7.03,+ 5585,-6 470
735,8.03,+ 13152,2 945
736,9.03,+ 14415,4 355
737,10.03,+ 13438,4 794


### Convert types

In [21]:
def is_string(value):
    return isinstance(value, str)
result = df.applymap(is_string)
print(result)

      Data  Nowe przypadki  Zmiana liczby aktywnych przypadków
0    False            True                                True
1    False            True                                True
2    False            True                                True
3    False            True                                True
4    False            True                                True
..     ...             ...                                 ...
734  False            True                                True
735  False            True                                True
736  False            True                                True
737  False            True                                True
738  False            True                                True

[739 rows x 3 columns]


C:\Users\Kamil\AppData\Local\Temp\ipykernel_8532\3434313055.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result = df.applymap(is_string)


In [22]:
df

,Data,Nowe przypadki,Zmiana liczby aktywnych przypadków
0,3.03,+ 0,+ 0
1,4.03,+ 1,+ 1
2,5.03,+ 0,+ 0
3,6.03,+ 4,+ 4
4,7.03,+ 1,+ 1
...,...,...,...
734,7.03,+ 5585,-6 470
735,8.03,+ 13152,2 945
736,9.03,+ 14415,4 355
737,10.03,+ 13438,4 794


In [23]:
print(df.dtypes)
typ_komorki = df.at[0, 'Nowe przypadki']

print("Typ komórki:", type(typ_komorki))
#convert columns 'Nowe przypadki' and 'Zmiana liczby aktywnych przypadków' to int
df['Nowe przypadki'] = df['Nowe przypadki'].str.replace(' ', '')
df['Zmiana liczby aktywnych przypadków'] = df['Zmiana liczby aktywnych przypadków'].str.replace(' ', '')

df['Nowe przypadki'] = df['Nowe przypadki'].astype(int)
df['Zmiana liczby aktywnych przypadków'] = df['Zmiana liczby aktywnych przypadków'].astype(int)


Data                                  float64
Nowe przypadki                         object
Zmiana liczby aktywnych przypadków     object
dtype: object
Typ komórki: <class 'str'>


In [35]:
df.tail()

,Data,Nowe przypadki,Zmiana liczby aktywnych przypadków
734,7.03,5585,-6470
735,8.03,13152,2945
736,9.03,14415,4355
737,10.03,13438,4794
738,11.03,11637,2917


# TODO: DOKOŃCZYĆ PREPROCESSING

In [27]:
# TODO: DOKOŃCZYĆ PREPROCESSING

## Visualisation

In [28]:
def visualise():
    pass

## Create Model

In [29]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, output_size, num_layers=2, hidden_size=64, num_heads=4, dropout=0.1):
        super(TransformerModel, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, src):
        src = src.permute(1, 0, 2)  # wymagane wymiary [seq_len, batch, input_size]
        output = self.transformer_encoder(src)
        output = self.fc(output[-1, :, :])  # ostatni element z sekwencji
        return output

## Training

In [30]:
data = df[['Data', 'Nowe przypadki', 'Zmiana liczby aktywnych przypadków']].copy()
data['Data'] = pd.to_datetime(data['Data'])
data.set_index('Data', inplace=True)

data = (data - data.mean()) / data.std()

train_data = data.iloc[:int(0.8 * len(data))]
test_data = data.iloc[int(0.8 * len(data)):]

X_train = torch.tensor(train_data.values, dtype=torch.float32)
X_test = torch.tensor(test_data.values, dtype=torch.float32)

In [31]:
def train_and_evaluate(model, epochs, criterion, optimizer):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, X_train)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    model.eval()
    with torch.no_grad():
        predicted = model(X_test)
        predicted = predicted.detach().numpy()

## Evaluation

In [32]:
input_size = 0
output_size = 0
num_layers = 0
hidden_size = 0
num_heads = 0
dropout = 0

model = TransformerModel(input_size, output_size, num_layers, hidden_size, num_heads, dropout)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100

train_and_evaluate(model, epochs, criterion, optimizer)

ValueError: embed_dim and num_heads must be greater than 0, got embed_dim=0 and num_heads=0 instead

## Conclusion

In [ ]:
# TODO: Do testowania proszę użyć kilku podejść podziału danych na uczące- testujace:
# TODO: (https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/)
# TODO: -K-Fold, Time SeriesSplit, Blocking Time Series Split
# TODO: Dla każdego wyniku stosujemy różne miary błędu (ewaluacji) MSE, MAE, R^2,  specyficzność czułość, index of agreement itp..